In [6]:
import numpy as np
import pandas as pd
from library.constants import *
import math



SIMPLIFIED_JOINTS = ["Head", "Neck", "Left Upper Arm", "Right Upper Arm", "Left Forearm", "Right Forearm",
                     "Left Hand", "Right Hand", "Pelvis", "Left Upper Leg", "Right Upper Leg",
                     "Left Lower Leg", "Right Lower Leg", "Left Foot", "Right Foot"]

SKELETON_CONNECTION_MAP = [["Head", "Neck"],
                           ["Neck", "Left Upper Arm"],
                           ["Left Upper Arm", "Left Forearm"],
                           ["Left Forearm", "Left Hand"],
                           ["Neck", "Right Upper Arm"],
                           ["Right Upper Arm", "Right Forearm"],
                           ["Right Forearm", "Right Hand"],
                           ["Neck", "Pelvis"],
                           ["Pelvis", "Left Upper Leg"],
                           ["Left Upper Leg", "Left Lower Leg"],
                           ["Left Lower Leg", "Left Foot"],
                           ["Pelvis", "Right Upper Leg"],
                           ["Right Upper Leg", "Right Lower Leg"],
                           ["Right Lower Leg", "Right Foot"]]

used_cols = []
for joints in SIMPLIFIED_JOINTS:
    used_cols.append(joints + " x")
    used_cols.append(joints + " y")
    used_cols.append(joints + " z")
used_cols += ["Frame"]

def read_data(file_name):
    return pd.read_excel(DATA_PATH + file_name,
                         sheet_name=RECORDING_TYPES,
                         usecols=used_cols)

# DATA_PATH = "C:\\Users\\gaoch\\MA\\Badminton weights\\P02_B\\"
# motion_recording = read_data("P02_B_noweightwithgloves.xlsx")

DATA_PATH = "C:\\Users\\gaoch\\MA\\Data\\P01_B_cut\\"
motion_recording = read_data("data.xlsx")

In [8]:
print(motion_recording)

{'Segment Position':       Frame  Pelvis x  Pelvis y  Pelvis z    Neck x    Neck y    Neck z  \
0         0  0.395517 -0.029583  1.094139  0.437497 -0.025474  1.563630   
1         1  0.395562 -0.029583  1.094110  0.437441 -0.025404  1.563610   
2         2  0.395610 -0.029584  1.094080  0.437388 -0.025336  1.563589   
3         3  0.395658 -0.029587  1.094051  0.437338 -0.025270  1.563569   
4         4  0.395708 -0.029590  1.094022  0.437288 -0.025204  1.563549   
...     ...       ...       ...       ...       ...       ...       ...   
9995   9995 -1.031869 -0.302918  1.101071 -1.003549 -0.296647  1.571541   
9996   9996 -1.032165 -0.301955  1.100987 -1.003646 -0.295565  1.571442   
9997   9997 -1.032457 -0.300992  1.100902 -1.003735 -0.294486  1.571342   
9998   9998 -1.032750 -0.300024  1.100817 -1.003818 -0.293413  1.571240   
9999   9999 -1.033041 -0.299055  1.100729 -1.003894 -0.292343  1.571136   

        Head x    Head y    Head z  ...  Right Foot z  Left Upper Leg x  \
0  

In [9]:
all_path = OUTPUT_FOLDER + "data.xlsx"
with pd.ExcelWriter(all_path) as writer:
    for recoding_type in RECORDING_TYPES:
        motion_recording[recoding_type][7600:7850].to_excel(writer, sheet_name=recoding_type, index=False)

In [ ]:
print(motion_recording.keys())